In [174]:
import numpy as np
import cv2

In [175]:
img = cv2.imread("images/Screenshot from 2023-09-30 18-15-58.png")
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow("grey", img)
cv2.waitKey(0)

-1

In [176]:
center_aruco_list = []
distance_from_rgb_list = []
angle_aruco_list = []
width_aruco_list = []
marker_id_list = []
ids = []

In [177]:
aruco_dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(img_grey, aruco_dictionary)

In [178]:
def calculate_rectangle_area(coordinates):
    '''
    Description:    Function to calculate area or detected aruco

    Args:
        coordinates (list):     coordinates of detected aruco (4 set of (x,y) coordinates)

    Returns:
        area        (float):    area of detected aruco
        width       (float):    width of detected aruco
    '''

    ############ Function VARIABLES ############

    # You can remove these variables after reading the instructions. These are just for sample.

    area = None
    width = None

    #My code
    coords = np.array(coordinates)
    v1,v2,v3,v4 = coords[1]-coords[0], coords[2]-coords[1], coords[3]-coords[2], coords[0]-coords[3]

    area = np.abs(0.5 * np.cross(v1, v2) + 0.5 * np.cross(v3, v4))
    width = np.sqrt(area)

    return area, width


In [179]:
area_threshold = 1000
corners_aruco_list = []

for i,I in enumerate(markerCorners):
    coords = I[0]   #I[0] is just syntax soup for greyscale images. it makes much more sense to think about it in an RGB way. I[0] means red, I[1] green and I[2] blue etc etc. But this isnt an RGB image so here its just I[0] means black and white.
    area, width = calculate_rectangle_area(coords)
    if area > area_threshold:
        center = np.zeros(2)
        for J in I[0]:
            center += J
        center /= 4
        center_aruco_list.append(center)
        width_aruco_list.append(width)
        marker_id_list.append(markerIds[i])
        corners_aruco_list.append(I)

print(marker_id_list, center_aruco_list, width_aruco_list, sep='\n')
        

[array([0], dtype=int32), array([0], dtype=int32)]
[array([797.25, 364.5 ]), array([259.25, 372.25])]
[384.66348929941347, 429.7220031601826]


In [180]:
markerCorners

(array([[[662.,  82.],
         [946., 118.],
         [929., 580.],
         [652., 678.]]], dtype=float32),
 array([[[ 92., 109.],
         [423.,  66.],
         [420., 719.],
         [102., 595.]]], dtype=float32))

In [181]:
corners_aruco_list = tuple(corners_aruco_list)

In [182]:
centers = np.array(center_aruco_list)
widths = np.array(width_aruco_list)
ids = np.array(marker_id_list)

In [183]:
cv2.aruco.drawDetectedMarkers(img, corners_aruco_list, ids)
cv2.imshow("img", img)
cv2.waitKey(0)

-1

In [184]:
cam_mat = np.array([[931.1829833984375, 0.0, 640.0], [0.0, 931.1829833984375, 360.0], [0.0, 0.0, 1.0]])
dist_mat = np.array([0.0,0.0,0.0,0.0,0.0])
size_of_aruco_m = 0.15

rvecs, tvecs, obj_points = cv2.aruco.estimatePoseSingleMarkers(corners_aruco_list, size_of_aruco_m, cam_mat, dist_mat)

print(rvecs)
print()
print(tvecs)

rvecs2 = rvecs.copy()
tvecs2 = -tvecs

[[[-2.80226703  0.04451172 -1.18767844]]

 [[-2.92821857 -0.07781272  0.91090604]]]

[[[ 0.05568862  0.00251957  0.28011792]]

 [[-0.11091453  0.00434848  0.25298085]]]


In [185]:
for i,I in enumerate(marker_id_list):
    try:
        cv2.drawFrameAxes(img, cam_mat, dist_mat, rvecs[i], tvecs[i], 0.25, 3)
    except Exception as e:
        print(e)

cv2.imshow("img2", img)
cv2.waitKey(0)

-1

In [186]:
img2 = img.copy()

for i,I in enumerate(marker_id_list):
    try:
        cv2.drawFrameAxes(img2, cam_mat, dist_mat, rvecs2[i], tvecs2[i], 0.25, 3)
    except Exception as e:
        print(e)

cv2.imshow("img2", img2)
cv2.waitKey(0)

-1

In [ ]:
def detect_aruco(image):
    '''
    Description:    Function to perform aruco detection and return each detail of aruco detected 
                    such as marker ID, distance, angle, width, center point location, etc.

    Args:
        image                   (Image):    Input image frame received from respective camera topic

    Returns:
        center_aruco_list       (list):     Center points of all aruco markers detected
        distance_from_rgb_list  (list):     Distance value of each aruco markers detected from RGB camera
        angle_aruco_list        (list):     Angle of all pose estimated for aruco marker
        width_aruco_list        (list):     Width of all detected aruco markers
        ids                     (list):     List of all aruco marker IDs detected in a single frame 
    '''

    ############ Function VARIABLES ############

    # ->  You can remove these variables if needed. These are just for suggestions to let you get started

    # Use this variable as a threshold value to detect aruco markers of certain size.
    # Ex: avoid markers/boxes placed far away from arm's reach position  
    area_threshold = 1500

    # The camera matrix is defined as per camera info loaded from the plugin used. 
    # You may get this from /camer_info topic when camera is spawned in gazebo.
    # Make sure you verify this matrix once if there are calibration issues.
    cam_mat = np.array([[931.1829833984375, 0.0, 640.0], [0.0, 931.1829833984375, 360.0], [0.0, 0.0, 1.0]])

    # The distortion matrix is currently set to 0. 
    # We will be using it during Stage 2 hardware as Intel Realsense Camera provides these camera info.
    dist_mat = np.array([0.0,0.0,0.0,0.0,0.0])

    # We are using 150x150 aruco marker size
    size_of_aruco_m = 0.15

    # You can remove these variables after reading the instructions. These are just for sample.
    center_aruco_list = []
    distance_from_rgb_list = []
    angle_aruco_list = []
    width_aruco_list = []
    marker_id_list = []
    corners_aruco_list = []
 
    ############ ADD YOUR CODE HERE ############

    # INSTRUCTIONS & HELP : 

    #	->  Convert input BGR image to GRAYSCALE for aruco detection
    image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_cpy = image.copy()

    #   ->  Use these aruco parameters-
    #       ->  Dictionary: 4x4_50 (4x4 only until 50 aruco IDs)
    aruco_dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4x4_50)

    #   ->  Detect aruco marker in the image and store 'corners' and 'ids'
    #       ->  HINT: Handle cases for empty markers detection. 
    markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(image_grey, aruco_dictionary)

    #   ->  Draw detected marker on the image frame which will be shown later
    cv2.aruco.drawDetectedMarkers(image_cpy, markerCorners, markerIds)

    #   ->  Loop over each marker ID detected in frame and calculate area using function defined above (calculate_rectangle_area(coordinates))
    #   ->  Remove tags which are far away from arm's reach positon based on some threshold defined
    for i,I in enumerate(markerCorners):
        coords = I[0]   #I[0] is just syntax soup for greyscale images. it makes much more sense to think about it in an RGB way. I[0] means red, I[1] green and I[2] blue etc etc. But this isnt an RGB image so here its just I[0] means black and white.
        area, width = calculate_rectangle_area(coords)
        if area > area_threshold:
            center = np.zeros(2)
            for J in I[0]:
                center += J
            center /= 4
            center_aruco_list.append(center)
            width_aruco_list.append(width)
            marker_id_list.append(markerIds[i])
            corners_aruco_list.append(I)

    #   ->  Calculate center points aruco list using math and distance from RGB camera using pose estimation of aruco marker
    #       ->  HINT: You may use numpy for center points and 'estimatePoseSingleMarkers' from cv2 aruco library for pose estimation
    rvecs, tvecs, obj_points = cv2.aruco.estimatePoseSingleMarkers(corners_aruco_list, size_of_aruco_m, cam_mat, dist_mat)
    
    #   ->  Draw frame axes from coordinates received using pose estimation
    #       ->  HINT: You may use 'cv2.drawFrameAxes
    for i,I in enumerate(marker_id_list):
        try:
            cv2.drawFrameAxes(image_cpy, cam_mat, dist_mat, rvecs[i], tvecs[i], 1, 1)
        except Exception as e:
            print(e)    

    ############################################

    return image_cpy, center_aruco_list, distance_from_rgb_list, angle_aruco_list, width_aruco_list, marker_id_list